In [10]:
import openai
import os
import re
from google.cloud import bigquery
import pandas as pd
import datetime

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv('OPENAI_API_KEY')
google_application_credentials = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")



In [17]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [18]:
# this is a fun way to test openai connection. can be skipped
messages =  [  
{'role':'system', 'content':'You are an assistant that explains SQL.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]
response = get_completion_from_messages(messages, temperature=2)
print(response)

To get away from old man McDonald (the farmer responsible for gathering his chickie ol' house :)|


In [19]:
def collect_messages():
    prompt = inp.value
    inp.value = ''
    context.append({'role': 'user1', 'content': f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': f"{response}"})


In [20]:
# function to read and concatenate files
def read_and_concatenate_files(path, file_list):
    concatenated_string = ''
    for file_name in file_list:
        file_path = os.path.join(path, file_name)
        with open(file_path, 'r') as file:
            content = file.read()
        concatenated_string += content
    return concatenated_string

#print("Combined content:")
#print(query_prompt_content)

# function to format prompt and response
def format_text(prompt, response):
    formatted_prompt = f"User: {prompt}"
    formatted_response = f"Assistant: {response}"
    return formatted_prompt, formatted_response

# function to extract SQL statement from response
def extract_sql(response):
    match = re.search(r'```\s*(.*?)\s*```', response, re.DOTALL)
    return match.group(1) if match else None

# function to execute SQL using BigQuery
def execute_query(sql):
    client = bigquery.Client()
    query_job = client.query(sql)
    
    return query_job.result()
    

In [21]:
#set context
path = 'prompts'
files = ['create_sql.txt','table_definition.json' ]
query_prompt_content = read_and_concatenate_files(path, files)


import ipywidgets as widgets
from IPython.display import display, clear_output


# Create input box and buttons
input_box = widgets.Text(description="Input:", layout= widgets.Layout(width='500px'))

button_conversation_gen = widgets.Button(description="Generate SQL")
button_conversation_exe = widgets.Button(description="Execute SQL")
output = widgets.Output()

# make formatted_response a global scope
formatted_prompt = ''
formatted_response = ''

# Function to handle button_conversation_gen click
def gen_sql(button):
    global formatted_prompt    # Use the global variable formatted_prompt
    global formatted_response  # Use the global variable formatted_response

    with output:
        clear_output(wait=True)
        prompt = input_box.value
        
        context = [{'role': 'system', 'content': f"{query_prompt_content}"}]  # accumulate messages
        context.append({'role': 'user', 'content': f"{prompt}"})
        response = get_completion_from_messages(context)
        formatted_prompt, formatted_response = format_text(prompt, response)
        print(formatted_prompt)
        print(formatted_response)

# Function to handle button_conversation_exe click
def exe_sql(button):
    with output:
        clear_output(wait=True)
       
        sql = extract_sql(formatted_response)
        if sql:
            try:
                results = execute_query(sql)
                print (formatted_response)
                print("\nResults:")
                # Print the column names
                print(" | ".join([field.name for field in results.schema]))
    
                # Print the results
                for row in results:
                    print(" | ".join([str(value) for value in row]))
            except Exception as e:
                print(f"An error occurred while executing the query: {e}")
        else:
            print("No SQL statement found in the response.")


# Connect the functions to button clicks
button_conversation_gen.on_click(gen_sql)
button_conversation_exe.on_click(exe_sql)

# Display input box, buttons and output
current_timestamp = datetime.datetime.now()
input_box.value=current_timestamp.strftime("%Y-%m-%d %H:%M:%S")

input_box_container = widgets.Box([input_box], layout=widgets.Layout(display='flex', justify_content='center'))
buttons_container = widgets.Box([button_conversation_gen, button_conversation_exe],
                                layout=widgets.Layout(display='flex', justify_content='center'))
display(input_box_container)
display(buttons_container)
display(output)


Box(children=(Text(value='2023-05-09 00:47:03', description='Input:', layout=Layout(width='500px')),), layout=…

Box(children=(Button(description='Generate SQL', style=ButtonStyle()), Button(description='Execute SQL', style…

Output()